In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
import optuna
import optuna.visualization as vis
    
n = 15
    
X = np.load('Datasets/kryptonite-%s-X.npy'%(n))
y = np.load('Datasets/kryptonite-%s-y.npy'%(n))

def objective(trial):

    if trial.should_prune():
        raise optuna.TrialPruned()

    ### ========== Shuffle and Split Data ========== ###

    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.6, random_state=42)  # 60% training
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)  # 20% validation, 20% test

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)
    X_test_scaled = scaler.transform(X_test)

    use_pca = trial.suggest_categorical('use_pca', ['True', 'False'])
    n_components_pca = trial.suggest_int('n_pca_components', 1, 20)


    ### ========== Train and Evaluate Model ========== ###

    if use_pca == 'True':
        pca = PCA(
            n_components=min(
                X.shape[1],
                n_components_pca
            )
        )
        X_train_scaled = pca.fit_transform(X_train_scaled)
        X_val_scaled = pca.transform(X_val)
        X_test_scaled = pca.transform(X_test)

    classifier = KNeighborsClassifier(
        n_neighbors=trial.suggest_int('n_neighbors', 5, 50),
        weights=trial.suggest_categorical('weights', ['uniform', 'distance']),
        n_jobs=-1
    )

    # classifier = KNeighborsClassifier(
    #     n_neighbors=20,
    #     weights='distance',
    #     n_jobs=-1
    # )

    classifier.fit(X_train_scaled, y_train)
    # print("Fit Model")

    # Evaluate on the validation set
    y_val_pred = classifier.predict(X_val_scaled)
    val_accuracy = accuracy_score(y_val, y_val_pred)
    # print(f"Validation Accuracy: {val_accuracy:.4f}")

    # Evaluate on the test set
    y_test_pred = classifier.predict(X_test_scaled)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    print(f"Test Accuracy: {test_accuracy:.4f}")

    return val_accuracy


# Create a study object
pruner = optuna.pruners.MedianPruner()
study = optuna.create_study(direction="maximize", pruner=pruner)

# Optimize the objective function
study.optimize(objective, n_trials=100)

print("Best hyperparameters:", study.best_params)
print("Best value:", study.best_value)

vis.plot_optimization_history(study).show()
vis.plot_param_importances(study).show()
vis.plot_slice(study).show()

# vis.plot_intermediate_values(study)

[I 2024-11-07 19:52:36,658] A new study created in memory with name: no-name-62b01b31-739d-4784-9e99-ec067aa6a78b
[I 2024-11-07 19:52:37,340] Trial 0 finished with value: 0.4612962962962963 and parameters: {'use_pca': 'False', 'n_pca_components': 19, 'n_neighbors': 29, 'weights': 'uniform'}. Best is trial 0 with value: 0.4612962962962963.


Test Accuracy: 0.4641


[I 2024-11-07 19:52:37,803] Trial 1 finished with value: 0.9572222222222222 and parameters: {'use_pca': 'False', 'n_pca_components': 5, 'n_neighbors': 9, 'weights': 'uniform'}. Best is trial 1 with value: 0.9572222222222222.


Test Accuracy: 0.9619


[I 2024-11-07 19:52:38,610] Trial 2 finished with value: 0.5175925925925926 and parameters: {'use_pca': 'True', 'n_pca_components': 16, 'n_neighbors': 46, 'weights': 'distance'}. Best is trial 1 with value: 0.9572222222222222.


...

Test Accuracy: 0.9622
Test Accuracy: 0.9620


[I 2024-11-07 19:53:01,946] Trial 97 finished with value: 0.9575925925925926 and parameters: {'use_pca': 'False', 'n_pca_components': 16, 'n_neighbors': 8, 'weights': 'distance'}. Best is trial 22 with value: 0.9575925925925926.


Test Accuracy: 0.9620


[I 2024-11-07 19:53:02,151] Trial 98 finished with value: 0.9568518518518518 and parameters: {'use_pca': 'False', 'n_pca_components': 19, 'n_neighbors': 14, 'weights': 'distance'}. Best is trial 22 with value: 0.9575925925925926.
[I 2024-11-07 19:53:02,259] Trial 99 finished with value: 0.9575925925925926 and parameters: {'use_pca': 'False', 'n_pca_components': 17, 'n_neighbors': 10, 'weights': 'distance'}. Best is trial 22 with value: 0.9575925925925926.


Test Accuracy: 0.9619
Test Accuracy: 0.9622
Best hyperparameters: {'use_pca': 'False', 'n_pca_components': 7, 'n_neighbors': 10, 'weights': 'distance'}
Best value: 0.9575925925925926
